In [ ]:
#r "nuget: Azure.AI.Projects, 1.0.0-beta.4"
#r "nuget: Azure.Identity"
//#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.40.0-preview"
//#r "nuget: Microsoft.SemanticKernel.Agents.AzureAI, 1.40.0-preview"

Installed Packages Azure.AI.Projects, 1.0.0-beta.4 Azure.Identity, 1.13.2

In [ ]:
using Azure;
using Azure.AI.Projects;
using Azure.Identity;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.AzureAI;

using System;
using Agent = Azure.AI.Projects.Agent;


In [ ]:
var credential = new DefaultAzureCredential();
string connectionString = "<REPLACE WITH YOUR PROJECT CONNECTION STRING>";
var client = new AgentsClient(connectionString, credential);


In [ ]:
// Get the agent id from Azure Hub Portal
string agentId = "<REPLACE WITH AGENT ID>";

// Here were are getting the created agent. But you can also create a new agent using CreateAgentAsync method.
Agent agent = await client.GetAgentAsync(agentId);

In [ ]:
// create thread and add message to the thread
AgentThread thread = await client.CreateThreadAsync();

// Add user message to the thread
Response<ThreadMessage> messageResponse = await client.CreateMessageAsync(
            thread.Id,
            MessageRole.User,
            "How are earnings looks like as compared to last year for Microsoft?");
        ThreadMessage message = messageResponse.Value;


In [ ]:
// code reference - https://learn.microsoft.com/en-us/azure/ai-services/agents/quickstart?pivots=programming-language-csharp

Response<PageableList<ThreadMessage>> messagesListResponse = await client.GetMessagesAsync(thread.Id);

Response<ThreadRun> runResponse = await client.CreateRunAsync(
            thread.Id,
            agent.Id,
            additionalInstructions: "");
        ThreadRun run = runResponse.Value;

        do
        {
            await Task.Delay(TimeSpan.FromMilliseconds(500));
            runResponse = await client.GetRunAsync(thread.Id, runResponse.Value.Id);
        }
        while (runResponse.Value.Status == RunStatus.Queued
            || runResponse.Value.Status == RunStatus.InProgress);

        Response<PageableList<ThreadMessage>> afterRunMessagesResponse
            = await client.GetMessagesAsync(thread.Id);
        IReadOnlyList<ThreadMessage> messages = afterRunMessagesResponse.Value.Data;

        foreach (ThreadMessage threadMessage in messages)
        {
            Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
            foreach (MessageContent contentItem in threadMessage.ContentItems)
            {
                if (contentItem is MessageTextContent textItem)
                {
                    Console.Write(textItem.Text);
                }
                else if (contentItem is MessageImageFileContent imageFileItem)
                {
                    Console.Write($"<image from ID: {imageFileItem.FileId}");
                }
                Console.WriteLine();
            }
        }

2025-03-03 17:16:23 -  assistant: For Microsoft, the comparison of their earnings from the previous fiscal year to the most recent fiscal year shows robust growth across several key financial metrics:

1. **Revenue** increased by $33.2 billion or 16%, from $211.9 billion in fiscal year 2023 to $245.1 billion in fiscal year 2024. This growth was driven by expansions in all primary segments, particularly Intelligent Cloud revenue driven by Azure and Productivity and Business Processes revenue driven by Office 365 Commercial【4:0†source】【4:18†source】.

2. **Gross Margin** increased by $25.0 billion or 17%, climbing from $146.1 billion to $171.0 billion【4:0†source】.

3. **Operating Income** rose by $20.9 billion or 24%, from $88.5 billion to $109.4 billion【4:0†source】【4:18†source】.

4. **Net Income** grew by 22%, surging from $72.4 billion to $88.1 billion【4:0†source】【4:18†source】.

5. **Diluted Earnings Per Share (EPS)** increased by 22%, from $9.68 to $11.80 per share【4:0†source】【4:18†sou